In [50]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE BangPatterns #-}

In [106]:
import Data.Text (Text)
import qualified Data.Text as T
import qualified Data.Text.IO as TIO

import Text.Megaparsec hiding (State)
import qualified Text.Megaparsec.Lexer as L
import Text.Megaparsec.Text (Parser)
import qualified Control.Applicative as CA

import qualified Data.Map.Strict as M
import Data.Map.Strict ((!))

import Data.Vector ((!), (//))
import qualified Data.Vector as V

import Data.List 

import qualified Data.Set as S

In [52]:
type Vec = V.Vector Integer

data Particle = Particle 
                    { position :: Vec
                    , velocity :: Vec
                    , acceleration :: Vec
                    } deriving (Show, Eq)

In [53]:
sc :: Parser ()
sc = L.space (skipSome spaceChar) CA.empty CA.empty

lexeme  = L.lexeme sc

integer       = lexeme L.integer
signedInteger = L.signed sc integer

symbol = L.symbol sc
separator = symbol ", "
comma = symbol ","

particlesP = particleP `sepBy` space
particleP = particlify <$> (symbol "p=" *> vecP <* separator)
                       <*> (symbol "v=" *> vecP <* separator)
                       <*> (symbol "a=" *> vecP)
    where particlify p v a = Particle {position = p, velocity = v, acceleration = a}


vecP = V.fromList <$> between (symbol "<") (symbol ">") (signedInteger `sepBy` comma)


successfulParse :: Text -> [Particle]
successfulParse input = 
        case parse particlesP "input" input of
                Left  _error -> [] -- TIO.putStr $ T.pack $ parseErrorPretty err
                Right instructions  -> instructions

In [54]:
parseTest vecP "<-833,-499,-1391>"

[-833,-499,-1391]

In [55]:
parseTest particleP "p=< 3,0,0>, v=<2,0,0>, a=<-1,0,0>"

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-1,0,0]}

In [56]:
parseTest particleP "p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>"

Particle {position = [4,0,0], velocity = [0,0,0], acceleration = [-2,0,0]}

In [57]:
parseTest particlesP "p=< 3,0,0>, v=<2,0,0>, a=<-1,0,0>\np=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>"

[Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-1,0,0]},Particle {position = [4,0,0], velocity = [0,0,0], acceleration = [-2,0,0]}]

In [58]:
(p0:p1:_) = successfulParse "p=< 3,0,0>, v=<2,0,0>, a=<-1,0,0>\np=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>"

In [59]:
p0

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-1,0,0]}

In [60]:
p1

Particle {position = [4,0,0], velocity = [0,0,0], acceleration = [-2,0,0]}

In [61]:
step :: Particle -> Particle
step particle = particle {position = p', velocity = v'}
    where pv' = V.zipWith3 updatePV (position particle) (velocity particle) (acceleration particle)
          !(p', v') = V.unzip pv'
          updatePV p v a = (p + v + a, v + a)

In [62]:
step $ step p1

Particle {position = [-2,0,0], velocity = [-4,0,0], acceleration = [-2,0,0]}

In [63]:
p2 = p0 {acceleration = V.fromList [-2, 0, 1]}
p2

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-2,0,1]}

In [64]:
step $ step p2

Particle {position = [1,0,3], velocity = [-2,0,2], acceleration = [-2,0,1]}

In [65]:
quiescent :: Particle -> Bool
quiescent particle = and qDimensions
    where qDimensions = V.zipWith3 sameSigns (position particle) (velocity particle) (acceleration particle)
          sameSigns !p !v !a = if a == 0 && v == 0
                               then True
                               else if a == 0
                                    then signum p == signum v
                                    else signum p == signum v && signum v == signum a

In [66]:
quiescent p2

False

In [67]:
simulate :: [Particle] -> [Particle]
simulate particles = 
    if all quiescent particles
    then particles
    else simulate (map step particles)

In [68]:
particles = [p0, p1, p2]
particles

[Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-1,0,0]},Particle {position = [4,0,0], velocity = [0,0,0], acceleration = [-2,0,0]},Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-2,0,1]}]

In [69]:
simulate particles

[Particle {position = [-2,0,0], velocity = [-3,0,0], acceleration = [-1,0,0]},Particle {position = [-26,0,0], velocity = [-10,0,0], acceleration = [-2,0,0]},Particle {position = [-17,0,15], velocity = [-8,0,5], acceleration = [-2,0,1]}]

In [70]:
pAbs :: Particle -> Particle
pAbs particle = particle {position = p', velocity = v', acceleration = a'}
    where !p' = V.map abs (position particle)
          !v' = V.map abs (velocity particle)
          !a' = V.map abs (acceleration particle)
    

In [71]:
p2

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-2,0,1]}

In [72]:
pAbs p2

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [2,0,1]}

In [73]:
pMin p1 p2 = Particle {position = p', velocity = v', acceleration = a'}
    where !p' = V.zipWith min (position p1) (position p2)
          !v' = V.zipWith min (velocity p1) (velocity p2)
          !a' = V.zipWith min (acceleration p1) (acceleration p2)

In [74]:
p0

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-1,0,0]}

In [75]:
p2

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [-2,0,1]}

In [76]:
pMin (pAbs p0) (pAbs p2)

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [1,0,0]}

In [77]:
pAbs p2

Particle {position = [3,0,0], velocity = [2,0,0], acceleration = [2,0,1]}

In [78]:
clearClosest :: [Particle] -> Maybe Int
clearClosest particles = elemIndex targetParticle absParticles
    where absParticles = map pAbs particles
          targetParticle = foldl1' pMin absParticles

In [79]:
pAbsA :: Particle -> Integer
pAbsA particle = V.foldl1' (+) $ V.map abs (acceleration particle)  

In [80]:
pAbsX :: Particle -> Integer
pAbsX particle = V.foldl1' (+) $ V.map abs (position particle)  

In [81]:
lowestAcc particles = elemIndices minAcc absAccs
    where absAccs = map pAbsA particles
          minAcc = minimum absAccs

In [82]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent20.txt"
        let particles = successfulParse text
--         print particles
        print $ lowestAcc particles
--         print $ part2 instrs

In [83]:
main

[21,457]

In [84]:
-- particles!!21

In [85]:
-- particles!!457

In [86]:
withMinX particles = minX `elemIndices` absXs
    where absXs = map pAbsX particles
          minX = minimum absXs

In [87]:
simulateQ :: [Particle] -> [Particle]
simulateQ particles = 
    if all quiescent particles
    then particles
    else simulateQ (map step particles)

In [88]:
simulate :: [Particle] -> [Particle]
simulate particles = 
    if all quiescent particles && length withMinXs == 1
    then particles
    else simulate (map step particles)
    where withMinXs = withMinX particles

In [89]:
pf = simulate particles

In [90]:
withMinX pf

[0]

In [100]:
-- part1 particles = head minInContext
--         where pf = simulate slowParticles
--               slowIndices = lowestAcc particles
--               slowParticles = map (particles !!) slowIndices
--               minInContext = map (slowIndices !!) $ withMinX pf

In [103]:
part1 particles = head $ withMinX pf
        where pf = simulate particles

In [104]:
main :: IO ()
main = do 
        text <- TIO.readFile "../../data/advent20.txt"
        let particles = successfulParse text
        print $ part1 particles
--         print $ part2 instrs

In [93]:
-- main

In [94]:
text <- TIO.readFile "../../data/advent20.txt"
particles = successfulParse text
particles!!0

Line 1: Use head
Found:
particles !! 0
Why not:
head particles

Particle {position = [-833,-499,-1391], velocity = [84,17,61], acceleration = [-4,1,1]}

In [96]:
simulateQ $ map (particles !!) $ lowestAcc particles

[Particle {position = [-59,74,-188], velocity = [-29,97,-150], acceleration = [-1,0,0]},Particle {position = [98,72,-533], velocity = [37,12,-172], acceleration = [0,1,0]}]

In [97]:
simulate $ map (particles !!) $ lowestAcc particles

[Particle {position = [-59,74,-188], velocity = [-29,97,-150], acceleration = [-1,0,0]},Particle {position = [98,72,-533], velocity = [37,12,-172], acceleration = [0,1,0]}]

In [105]:
main

457

In [116]:
removeColliders particles = particles'
    where positions = map position particles
          duplicatePositions = S.fromList $ concat $ filter (\g -> length g > 1) $ group $ sort positions
          particles' = filter (\p -> not (S.member (position p) duplicatePositions)) particles

In [117]:
simulateC :: Integer -> [Particle] -> [Particle]
simulateC 0 particles = particles
simulateC t particles = simulateC (t - 1) (map step particles')
    where particles' = removeColliders particles

In [118]:
part2 n particles = length $ simulateC n particles

In [119]:
length particles

1000

In [135]:
part2 35 particles

570

In [137]:
[(n, part2 n particles) | n <- [10..45]]

[(10,1000),(11,979),(12,979),(13,973),(14,955),(15,932),(16,921),(17,906),(18,874),(19,858),(20,831),(21,821),(22,809),(23,795),(24,791),(25,771),(26,752),(27,723),(28,703),(29,669),(30,648),(31,634),(32,622),(33,617),(34,589),(35,570),(36,542),(37,522),(38,494),(39,481),(40,448),(41,448),(42,448),(43,448),(44,448),(45,448)]